In [1]:
import numpy as np 

def load_dataset(filename):
    data_mat = []; label_mat = []
    with open(filename) as fr:
        for line in fr.readlines():
            line_arr = line.strip().split('\t')
            data_mat.append([float(line_arr[0]), float(line_arr[1])])
            label_mat.append(float(line_arr[2]))
    return data_mat, label_mat

def select_jrand(i, m):
    return np.random.randint(i, m)

def clip_alpha(aj, H, L):
    aj = max(aj, L)
    aj = min(aj, H)
    return aj

In [8]:
data_arr, label_arr = load_dataset('testSet.txt')
print(label_arr)

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


In [27]:
def smo_simple(data, classlabels, C, toler, max_iter):
    """
    C:常数
    toler:容错率
    """
    data_mat = np.mat(data)
    label_mat = np.mat(classlabels).transpose()
    b = 0; m,n = np.shape(data_mat)
    alphas = np.mat(np.zeros((m, 1)))
    num = 0
    while (num < max_iter):
        alpha_pair_changed = 0 
        for i in range(m):
            fXi = float(np.multiply(alphas, label_mat).T * \
                (data_mat * data_mat[i, :].T)) + b  # 计算出的类别
            Ei = fXi - float(label_mat[i])  # 误差Ei，根据此优化
            if ((label_mat[i] * Ei < toler) and (alphas[i] < C)) or \
                ((label_mat[i] * Ei > toler) and (alphas[i] > 0) and \
                    (alphas[i] > 0)):   # 如果alpha可以改更改进入优化过程
                j = select_jrand(i, m)  # 随机选择第二个alpha
                fXj = float(np.multiply(alphas, label_mat).T * \
                    (data_mat * data_mat[j, :].T)) + b 
                Ej = fXj - float(label_mat[j])
                alpha_i_hold = alphas[i].copy()
                alpha_j_hold = alphas[j].copy()
                if (label_mat[i] != label_mat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H: 
                    # print("L==H")
                    continue
                eta = 2.0 * data_mat[i,:] * data_mat[i,:].T - \
                    data_mat[i,:]*data_mat[i,:].T - data_mat[j,:]*data_mat[j,:].T   # eta是alpha[j]的最优修改量
                if eta >=0: 
                    # print("eta>=0")
                    continue 
                alphas[j] -= label_mat[j]*(Ei - Ej)/eta
                alphas[j] = clip_alpha(alphas[j], H, L)
                if (abs(alphas[j] - alpha_j_hold) < 0.00001):
                    # print("j not moving enough")
                    continue
                alphas[i] += label_mat[j] * label_mat[i] * (alpha_j_hold - alphas[j])   # i的修改方向和j相反
                b1 = b - Ei \
                    - label_mat[i] * (alphas[i] - alpha_i_hold) * data_mat[i,:] * data_mat[i,:].T \
                    - label_mat[j] * (alphas[j] - alpha_j_hold) * data_mat[i,:] * data_mat[j,:].T
                b2 = b - Ej \
                    - label_mat[i] * (alphas[i] - alpha_i_hold) * data_mat[i,:] * data_mat[j,:].T \
                    - label_mat[j] * (alphas[j] - alpha_j_hold) * data_mat[j,:] * data_mat[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1 
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2 
                else:
                    b = (b1 + b2) / 2.0
                alpha_pair_changed += 1
                # print("iter : %d i:%d, pairs changed %d" % \
                #     (num, i, alpha_pair_changed))
        if (alpha_pair_changed == 0): 
            num += 1
        else: 
            num = 0
        # print("iteration number: %d" % num)
    return b, alphas

In [31]:
b, alphas = smo_simple(data_arr, label_arr, 0.6, 0.001, 40)

matrix([[-3.886982]])

In [32]:
b

matrix([[-3.886982]])

In [33]:
alphas[alphas>0]

matrix([[1.26943232e-01, 2.47039779e-01, 3.46944695e-17, 3.73990223e-01]])

In [35]:
for i in range(100):
    if alphas[i] > 0.0:
        print(data_arr[i], label_arr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[8.804461, 3.044301] 1.0
[6.080573, 0.418886] 1.0


In [36]:
# 利用完整的platt SMO 优化算法

In [41]:
class DataStruct:
    def __init__(self, data_mat, classlabels, C, toler):
        self.X = data_mat
        self.label_mat = classlabels
        self.C = C 
        self.tol = toler 
        self.m = np.shape(data_mat)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.e_cache = np.mat(np.zeros((self.m, 2)))


def calc_ek(ds, k):
    fXk = float(np.multiply(ds.alphas, ds.label_mat).T *\
        (ds.X * ds.X[k,:].T)) + ds.b
    Ek = fXk - float(ds.label_mat[k])
    return Ek

def select_j(i, ds, Ei):
    max_k = -1
    max_deltaE = 0
    Ej = 0
    ds.e_cache[i] = [i, Ei]
    valid_Ecache_lst = np.nonzero(np.array(ds.e_cache[:, 0]))[0]
    if (len(valid_Ecache_lst) > 1):
        for k in valid_Ecache_lst:
            if k == i:
                continue
            Ek = calc_ek(ds, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > max_deltaE):
                max_k = k
                max_deltaE = deltaE 
        return max_k, Ej 
    else:
        j = select_jrand(i, ds.m)
        Ej = calc_ek(ds, j)
    return j, Ej 


def updateEk(ds, k):
    Ek = calc_ek(ds, k)
    ds.e_cache[k] = [1, Ek]

In [52]:
def innerL(i, ds):
    Ei = calc_ek(ds, i)
    if ((ds.label_mat[i] * Ei < -ds.tol) and (ds.alphas[i] < ds.C)) or \
        ((ds.label_mat[i] * Ei > ds.tol) and (ds.alphas[i] > 0)):
        j, Ej = select_j(i, ds, Ei)
        alpha_i_old = ds.alphas[i].copy()
        alpha_j_old = ds.alphas[j].copy()
        if (ds.label_mat[i] != ds.label_mat[j]):
            L = max(0, ds.label_mat[j] - ds.label_mat[i])
            H = min(ds.C, ds.C + ds.label_mat[j] - ds.label_mat[i])
        else:
            L = max(0, ds.label_mat[j] + ds.label_mat[i] - ds.C)
            H = min(ds.C, ds.label_mat[j] + ds.label_mat[i])
        if L == H:
            # print("L==H")
            return 0 
        eta = 2.0 * ds.X[i,:] * ds.X[i,:].T - \
                    ds.X[i,:]*ds.X[i,:].T - ds.X[j,:]*ds.X[j,:].T
        if eta >=0: 
            # print("eta>=0")
            return 0 
        ds.alphas[j] -= ds.label_mat[j]*(Ei - Ej)/eta
        ds.alphas[j] = clip_alpha(ds.alphas[j], H, L)
        updateEk(ds, j)
        if (abs(alphas[j] - alpha_j_old) < 0.00001):
            # print("j not moving enough")
            return 0
        ds.alphas[i] += ds.label_mat[j] * ds.label_mat[i] * (alpha_j_old - ds.alphas[j])
        updateEk(ds, i)
        b1 = ds.b - Ei \
            - ds.label_mat[i] * (ds.alphas[i] - alpha_i_old) * ds.X[i,:] * ds.X[i,:].T \
            - ds.label_mat[j] * (ds.alphas[j] - alpha_j_old) * ds.X[i,:] * ds.X[j,:].T
        b2 = ds.b - Ej \
            - ds.label_mat[i] * (ds.alphas[i] - alpha_i_old) * ds.X[i,:] * ds.X[j,:].T \
            - ds.label_mat[j] * (ds.alphas[j] - alpha_j_old) * ds.X[j,:] * ds.X[j,:].T
        if (0 < ds.alphas[i]) and (ds.C > ds.alphas[i]):
            ds.b = b1 
        elif (0 < ds.alphas[j]) and (ds.C > ds.alphas[j]):
            ds.b = b2 
        else:
            ds.b = (b1 + b2) / 2.0
        return 1 
    else:
        return 0

In [53]:
def smo_p(data_mat, classlabels, C, toler, max_iter, kTup=('lin', 0)):
    ds = DataStruct(np.mat(data_mat), np.mat(classlabels).transpose(), C, toler)
    num = 0
    entire_set = True 
    alpha_pairs_changed = 0
    while ((num < max_iter) and (alpha_pairs_changed > 0) or (entire_set)):
        alpha_pairs_changed = 0
        if entire_set:  # 遍历所有值
            for i in range(ds.m):
                alpha_pairs_changed += innerL(i, ds)
            print("Fullset, iter: %d i: %d, pairs changed %d" %\
                (num, i, alpha_pairs_changed))
            num += 1
        else:       # 遍历非边界值
            non_bounds = np.nonzero((np.array(ds.alphas) > 0) * (np.array(ds.alphas) < C))[0]
            for i in non_bounds:
                alpha_pairs_changed += innerL(i, ds)
                print("non-bound, iter: %d i: %d, pairs changed %d" %\
                    (num, i, alpha_pairs_changed))
                num += 1
        if entire_set:
            entire_set = False 
        elif alpha_pairs_changed == 0:
            entire_set = True 
        print("iteration number: %d" % num)
    return ds.b, ds.alphas

In [60]:
data_arr, label_arr = load_dataset('testSet.txt')
b, alphas = smo_p(data_arr, label_arr, 0.6, 0.001, 40)

<ipython-input-41-9b5ba94718f5>:23: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  ds.e_cache[i] = [i, Ei]
<ipython-input-41-9b5ba94718f5>:43: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  ds.e_cache[k] = [1, Ek]
Fullset, iter: 0 i: 99, pairs changed 8
iteration number: 1
iteration number: 1
Fullset, iter: 1 i: 99, pairs changed 1
iteration number: 2
iteration number: 2
Fullset, iter: 2 i: 99, pairs changed 1
iteration number: 3
iteration number: 3
Fullset, iter: 3 i: 99, pairs changed 1
iteration number: 

In [61]:
b

matrix([[-17.52136278]])

In [62]:
alphas[alphas>0]

matrix([[0.6]])